In [ ]:
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from   scipy.stats import norm

In [ ]:
DATAFRAME = '../metadata/kellinghaus_substages.csv'

In [ ]:
df_stages = pd.read_csv(DATAFRAME)
df_stages

In [ ]:
def set_up_age_space(start=10, stop=35, n_ages=2501, n_decimals=2):
    x = np.linspace(start=start, stop=stop, num=n_ages, dtype=np.float64)
    x = np.round(x, decimals=n_decimals)
    return x

In [ ]:
x_ages = set_up_age_space()
x_ages

In [ ]:
def stage_probability_density(df_stages, x_ages):
    
    stage_prob_dens = []

    # Loop through rows of the dataframe
    for row in df_stages.iterrows():

        row = row[1]

        # Extract mean and standard deviation
        mean = row['Mean']
        sd   = row['SD']
        
        # Sample probability density from PDF
        pd_sample = norm.pdf(x_ages, loc=mean, scale=sd)
        stage_prob_dens.append(pd_sample)
    
    stage_prob_dens = np.asarray(stage_prob_dens)
    
    return stage_prob_dens

In [ ]:
stage_prob_dens = stage_probability_density(df_stages, x_ages)

In [ ]:
def plot_stage_probability_density(stage_prob_dens, df_stages, x_ages):

    # Sanity checks
    if stage_prob_dens.shape[0] != df_stages.shape[0]:
        print('ERROR: Expected {:d} stage probability entries and got {:d}'.format(df_stages.shape[0], stage_prob_dens.shape[0]))
        return -1
    
    if stage_prob_dens.shape[1] != x_ages.shape[0]:
        print('ERROR: Expected stage probabilities to have {:d} entries and got {:d}'.format(x_ages.shape[0], stage_prob_dens.shape[1]))
        return -1
    
    fig, ax = plt.subplots(1, 2, figsize=(20, 10))

    ax[1].set_prop_cycle(color=cm.get_cmap('tab10').colors)

    for i, row in enumerate(df_stages.iterrows()):

        row = row[1]

        stage = row['Stage']
        sex   = row['Sex']
        mean  = row['Mean']
        sd    = row['SD']

        if sex == 'Female':
            ax[0].plot(x_ages, stage_prob_dens[i], label='stage {:s}'.format(str(stage)))
        elif sex == 'Male':
            ax[1].plot(x_ages, stage_prob_dens[i], label='stage {:s}'.format(str(stage)))

    for axis in ax:
        axis.set_ylim(0.00,0.46)
        axis.set_xlabel('age / (years)', fontsize=16)
        axis.set_ylabel('probability density', fontsize=16)
        axis.tick_params(labelsize=16, size=4)
        axis.legend(fontsize=16)
        
    ax[0].set_title('female', fontsize=18)
    ax[1].set_title('male', fontsize=18)

    plt.savefig('../results/plots/kh_stage_prob_dens.png', facecolor='white', bbox_inches='tight', dpi=200)
    plt.show()

In [ ]:
plot_stage_probability_density(stage_prob_dens, df_stages, x_ages)

In [ ]:
def stage_probabilities(query_age, stage_prob_dens, df_stages, x_ages, equalize_best_probs: bool = False):
        
    stage_probs = []

    # Calculate probability for the query age to be in stage 'i'
    for i in range(df_stages.shape[0]):

        try:
            x_pos               = np.where(x_ages==query_age)[0][0]
        except:
            raise ValueError('The query age {:.3f} could not be found in the age space'.format(query_age))
        
        prob_age_in_stage_i = stage_prob_dens[i][x_pos]
        stage_probs.append(prob_age_in_stage_i)

    if equalize_best_probs:
        stage_probs_male   = stage_probs[0::2]
        stage_probs_female = stage_probs[1::2]
        
        # Sort probabilites
        sorted_probs_m = np.argsort(stage_probs_male)
        sorted_probs_f = np.argsort(stage_probs_female)

        # Calculate average of the two highest probabilties
        avg_prob_m = (stage_probs_male[sorted_probs_m[-1]] + stage_probs_male[sorted_probs_m[-2]]) / 2.0
        avg_prob_f = (stage_probs_female[sorted_probs_f[-1]] + stage_probs_female[sorted_probs_f[-2]]) / 2.0

        # Replace the two highest probabilties with their average
        stage_probs_male[sorted_probs_m[-1]]   = avg_prob_m
        stage_probs_male[sorted_probs_m[-2]]   = avg_prob_m
        stage_probs_female[sorted_probs_m[-1]] = avg_prob_f
        stage_probs_female[sorted_probs_m[-2]] = avg_prob_f

        # "Zip" together both arrays
        stage_probs = []
        for x, y in zip(stage_probs_male,stage_probs_female):
            stage_probs.append(x)
            stage_probs.append(y)
        stage_probs = np.asarray(stage_probs)

    # Normalize probabilites
    sum_stage_probs_male   = np.sum(stage_probs[0::2])
    sum_stage_probs_female = np.sum(stage_probs[1::2])

    n_stages = int(df_stages.shape[0]/2.0)
    
    sum_stage_probs        = np.array([sum_stage_probs_male,sum_stage_probs_female]*n_stages)
    stage_probs            = np.divide(stage_probs, sum_stage_probs)
    
    return stage_probs

In [ ]:
query_age   = x_ages[1100]
stage_probs = stage_probabilities(query_age, stage_prob_dens, df_stages, x_ages, equalize_best_probs=False)

print('Query age = {:.2f} years'.format(query_age))
print('Stage probabilities (male)   = {} %'.format(stage_probs[0::2]*100))
print('Stage probabilities (female) = {} %'.format(stage_probs[1::2]*100))

fig, ax = plt.subplots(1, 2, figsize=(18, 9))

ax[0].bar(df_stages['Stage'].to_list()[::2], stage_probs[0::2]*100, color='cornflowerblue')
ax[1].bar(df_stages['Stage'].to_list()[::2], stage_probs[1::2]*100, color='cornflowerblue')

for axis in ax:
    axis.set_xlabel('stage', fontsize=18)
    axis.set_ylabel('probability / (%)', fontsize=18)
    axis.tick_params(labelsize=14, size=8)
ax[0].set_title('Male', fontsize=18)
ax[1].set_title('Female', fontsize=18)

plt.suptitle('Stage probabilites for age = {:.2f}'.format(query_age), fontsize=20)
plt.show()

In [ ]:
def most_likely_ossification_stage(query_age, sex, df_stages, stage_probs, verbose=False):
    
    stages   = df_stages['Stage'].to_numpy()
    stages_m = stages[0::2]
    stages_f = stages[1::2]
    
    if sex == 'Male':
        stage_id   = np.argmax(stage_probs[0::2])
        stage_prob = np.max(stage_probs[0::2])
        stage      = stages_m[stage_id]
    elif sex == 'Female':
        stage_id   = np.argmax(stage_probs[1::2])
        stage_prob = np.max(stage_probs[1::2])
        stage      = stages_f[stage_id]
    
    if verbose:
        print('Most likely ossification stage for age = {:.2f} years ({:s}) = stage {} ({:.1f} %)'.format(query_age, sex, stage, stage_prob*100))
    
    return stage

In [ ]:
most_likely_ossification_stage(query_age, 'Male', df_stages, stage_probs, verbose=True)

In [ ]:
def prediction_errors(query_age, df_stages, verbose=False):
    
    pred_errors = []
    
    if verbose:
        print('Query age:     {:.2f}\n'.format(query_age))
    
    for i, row in enumerate(df_stages.iterrows()):

        row = row[1]

        stage = row['Stage']
        sex   = row['Sex']
        mean  = row['Mean']
    
        # Calculate prediction error
        error = mean - query_age
        
        pred_errors.append(error)
    
        if verbose:
            print('Stage:         {:d}'.format(stage))
            print('Sex:           {:s}'.format(sex))
            print('Predicted age: {:.2f}'.format(mean))
            print('Error:         {:.2f}\n'.format(error))
            
    pred_errors = np.asarray(pred_errors)
    
    return pred_errors

In [ ]:
query_age   = x_ages[1100]
pred_errors = prediction_errors(query_age, df_stages=df_stages)

print('Query age = {:.2f} years'.format(query_age))
print('prediction errors (male)   = {}'.format(pred_errors[0::2]))
print('prediction errors (female) = {}'.format(pred_errors[1::2]))

fig, ax = plt.subplots(1, 2, figsize=(18, 9))

ax[0].bar(df_stages['Stage'].to_list()[::2], pred_errors[0::2], color='firebrick')
ax[1].bar(df_stages['Stage'].to_list()[::2], pred_errors[1::2], color='firebrick')

for axis in ax:
    axis.set_ylim(np.min(pred_errors)-1,np.max(pred_errors)+1)
    axis.set_xlabel('stage', fontsize=18)
    axis.set_ylabel('prediction error / (years)', fontsize=18)
    axis.tick_params(labelsize=14, size=8)
ax[0].set_title('Male', fontsize=18)
ax[1].set_title('Female', fontsize=18)

plt.suptitle('Stage prediction error for age = {:.2f}'.format(query_age), fontsize=20)

plt.show()

In [ ]:
query_age   = x_ages[1100]
pred_errors = prediction_errors(query_age, df_stages=df_stages)
stage_probs = stage_probabilities(query_age, stage_prob_dens, df_stages, x_ages)

weighted_pred_erros = np.multiply(stage_probs, pred_errors)

print('Query age = {:.2f} years'.format(query_age))
print('Sum of weighted prediction errors (male)   = {:.2f} years'.format(np.sum(weighted_pred_erros[0::2])))
print('Sum of weighted prediction errors (female) = {:.2f} years'.format(np.sum(weighted_pred_erros[1::2])))

fig, ax = plt.subplots(1, 2, figsize=(18, 9))

ax[0].bar(df_stages['Stage'].to_list()[::2], weighted_pred_erros[0::2], color='firebrick')
ax[1].bar(df_stages['Stage'].to_list()[::2], weighted_pred_erros[1::2], color='firebrick')

for axis in ax:
    axis.axhline(y=0, ls='--', lw=2, c='k')
    axis.set_ylim(np.min(weighted_pred_erros)-0.5,np.max(weighted_pred_erros)+0.5)
    axis.set_xlabel('stage', fontsize=18)
    axis.set_ylabel('weighted prediction errors / (years)', fontsize=18)
    axis.tick_params(labelsize=14, size=8)
ax[0].set_title('Male', fontsize=18)
ax[1].set_title('Female', fontsize=18)

plt.suptitle('Weighted stage error for age = {:.2f}'.format(query_age), fontsize=20)

plt.show()

### Calculate errors

In [ ]:
errors_m = []
errors_f = []

for age in x_ages:
    pred_errors         = prediction_errors(age, df_stages=df_stages)
    stage_probs         = stage_probabilities(age, stage_prob_dens, df_stages, x_ages)
    weighted_pred_erros = np.multiply(stage_probs, np.abs(pred_errors))
    
    absolut_error_age_m = np.abs(np.sum(weighted_pred_erros[0::2]))
    absolut_error_age_f = np.abs(np.sum(weighted_pred_erros[1::2]))
    
    errors_m.append(absolut_error_age_m)
    errors_f.append(absolut_error_age_f)

errors_m_equ = []
errors_f_equ = []

for age in x_ages:
    pred_errors         = prediction_errors(age, df_stages=df_stages)
    stage_probs         = stage_probabilities(age, stage_prob_dens, df_stages, x_ages, equalize_best_probs=True)
    weighted_pred_erros = np.multiply(stage_probs, np.abs(pred_errors))
    
    absolut_error_age_m = np.abs(np.sum(weighted_pred_erros[0::2]))
    absolut_error_age_f = np.abs(np.sum(weighted_pred_erros[1::2]))
    
    errors_m_equ.append(absolut_error_age_m)
    errors_f_equ.append(absolut_error_age_f)

In [ ]:
plt.figure(figsize=(10, 10))

plt.plot(x_ages, errors_f, ls='-',  lw=1.5, c='k', alpha=0.9, label='female')
plt.plot(x_ages, errors_m, ls='--', lw=1.5, c='k', alpha=0.9, label='male')

plt.ylim(0,4)
plt.tick_params(labelsize=16, size=4)
plt.xlabel('age / (years)', fontsize=16)
plt.ylabel('absolute error / (years)', fontsize=16)
plt.legend(fontsize=16)

plt.savefig('../results/plots/kh_ae_vs_age.png', facecolor='white', bbox_inches='tight', dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 8))

ax[0].scatter(x_ages, errors_m_equ)
ax[1].scatter(x_ages, errors_f_equ)

for axis in ax:
    axis.set_ylim(0,4)
    axis.set_xlabel('age / (years)', fontsize=18)
    axis.set_ylabel('mean absolute error / (years)', fontsize=18)
    axis.tick_params(labelsize=14, size=8)
ax[0].set_title('Male (equalized probabilities)', fontsize=18)
ax[1].set_title('Female (equalized probabilities)', fontsize=18)

plt.show()

In [ ]:
data_kh_mae_m = np.array([x_ages, errors_m]).swapaxes(0,1)
df_kh_mae_m   = pd.DataFrame(data=data_kh_mae_m, columns=['age', 'ae'])
df_kh_mae_m.to_csv('../results/kh_ae_male.csv', index=False)

data_kh_mae_m_equ = np.array([x_ages, errors_m_equ]).swapaxes(0,1)
df_kh_mae_m_equ   = pd.DataFrame(data=data_kh_mae_m_equ, columns=['age', 'ae'])
#df_kh_mae_m_equ.to_csv('../results/kh_ae_male_equ.csv', index=False)

df_kh_mae_m

In [ ]:
data_kh_mae_f = np.array([x_ages, errors_f]).swapaxes(0,1)
df_kh_mae_f   = pd.DataFrame(data=data_kh_mae_f, columns=['age', 'ae'])
df_kh_mae_f.to_csv('../results/kh_ae_female.csv', index=False)

data_kh_mae_f_equ = np.array([x_ages, errors_f_equ]).swapaxes(0,1)
df_kh_mae_f_equ   = pd.DataFrame(data=data_kh_mae_f_equ, columns=['age', 'ae'])
#df_kh_mae_f_equ.to_csv('../results/kh_ae_female_equ.csv', index=False)

df_kh_mae_f